<div style="position:relative; height:160px;">
<img src="logoDECATUFS.png" width="500" height="500" style="position:absolute; left:-8px;">
</div>

<div style="text-align:center;font-size:17px; white-space:pre;margin-top:-35px">
        <h3>             Modelos Lineares Generalizados</h3>
        Disciplina ofertada pelo DECAT/UFS
        Código: ESTAT0092
        Nível: Graduação
        Carga horária: 60h
        Período: 2020.2
        Professor responsável e ministrante: Luiz Henrique Dore
        <h3>            Aula 13: avaliação da significância individual dos coeficientes de regressão</h3>
</div>

# Exemplo 2

<p style="text-align: justify; font-size:16px">
Considere o conjunto de dados <b>trees.dat</b>, descrito no exemplo 4 da aula 2. Os objetivos são os seguintes: ajustar um modelo de regressão gama, com ligação logarítmica, no qual a variável resposta é o volume e as variáveis preditoras são a altura e o diâmetro; executar o teste de Wald, ao nível de 5% de significância, para avaliar a significância individual dos coeficientes de regressão; calcular o intervalo de confiança, com 95% de confiança, para cada um dos coeficientes de regressão.
<p>

### 2.1 Leitura e visualização dos dados

In [1]:
dados<-read.table("/Dados/Google_Drive_UFS/Aulas/2020_2/MLG/Dados/trees.dat",header=TRUE)
n<-dim(dados)[1] #Número de observações
head(dados) #Visualizando as primeiras observações

,Diametro,Altura,Volume
,<dbl>,<int>,<dbl>
1,8.3,70,10.3
2,8.6,65,10.3
3,8.8,63,10.2
4,10.5,72,16.4
5,10.7,81,18.8
6,10.8,83,19.7


In [2]:
diam<-dados[,1]
alt<-dados[,2]
vol<-dados[,3]

### 2.2 Formulação do modelo

<p style="text-align: justify; font-size:15px">
    <ol style="font-size:16px">
        <li>$Volume\sim Gama(a,b)$;</li>
        <li>$\eta=\beta_0+\beta_1\cdot Altura+\beta_2\cdot Diâmetro$;</li>
        <li>$\ln\left(\mu\right)=\eta$.</li>
    </ol>
</p>

### 2.3 Estimação dos parâmetros por máxima verossimilhança usando as funções ``glm`` e ``gamma.shape``.

<p style="text-align: justify;font-size: 16px">Carregando os pacotes necessários.</p>

In [18]:
library(MASS) #Contém a função gamma.shape

<p style="text-align: justify;font-size: 16px">Estimação dos coeficientes de regressão.</p>

In [19]:
aj<-glm(formula=Volume~Diametro+Altura,family=Gamma(link="log"),data=dados)
beta<-aj$coefficients

<p style="text-align: justify;font-size: 16px">Estimação do parâmetro de dispersão.</p>

In [20]:
ajPhi<-gamma.shape(aj)
alpha<-ajPhi$alpha
phi<-1/alpha
names(phi)<-"Dispersão"

<p style="text-align: justify;font-size: 16px">Visualizando as estimativas dos parâmetros.</p>

In [21]:
print(c(beta,phi))

(Intercept)    Diametro      Altura   Dispersão 
0.092304581 0.145281192 0.016577883 0.008455011 


### 2.4 Executando o teste de Wald e calculando os intervalos de confiança manualmente

<p style="text-align: justify; font-size:16px">Gerando o vetor dos valores observados da resposta e a matriz design.</p>

In [7]:
y<-vol
X<-cbind(diam,alt)
X<-cbind(1,X)
colnames(X)<-c("","Diâmetro","Altura")
head(X)

,Diâmetro,Altura
1,8.3,70
1,8.6,65
1,8.8,63
1,10.5,72
1,10.7,81
1,10.8,83


<p style="text-align: justify; font-size:16px">A estatística de Wald é $T_{W_j}=\frac{\hat\beta_j}{\sqrt{\widehat{Var(\beta_j)}}}$, onde $\widehat{Var(\hat\beta_j)}$ é o elemento $j+1$ da diagonal de $\boldsymbol{K}^{\ \ \ -1}_{\boldsymbol{\widehat\beta}\boldsymbol{\widehat\beta}}$. No caso da distribuição Gama com ligação logarítmica, pode-se mostrar que $\boldsymbol{K}_{\boldsymbol{\beta}\boldsymbol{\beta}}=\phi^{-1}X^TX$ (ver exemplo 4 da aula 5). Logo, $\boldsymbol{K}^{\ \ \ -1}_{\boldsymbol{\widehat\beta}\boldsymbol{\widehat\beta}}=\hat\phi (X^TX)^{-1}$.</p>

<p style="text-align: justify;font-size: 16px">Calculando $Var(\beta_j)$, para cada $j\in\{0,1,\cdots,p\}$. O vetor <font face="'Courier New'">betaVar</font> contém as variâncias calculadas.</p>

In [8]:
Kbb<-phi^{-1}*t(X)%*%X #Calcuando Kbb
KbbInv<-solve(Kbb) #Calcuando a inversa de Kbb
betaVar<-diag(KbbInv) #Recolhendo a diagonal da inversa de Kbb, cujos valores correspondem às variâncias

<p style="text-align: justify;font-size: 16px">Visualizando os valores de $Var(\beta_j)$, para cada $j\in\{0,1,\cdots,p\}$.</p>

In [9]:
print(betaVar)

                 Diâmetro       Altura 
4.186873e-02 3.918490e-05 9.504668e-06 


<p style="text-align: justify;font-size: 16px">Calculando e visualizando os valores de $T_{W_j}$, para cada $j\in\{0,1,\cdots,p\}$.</p>

In [10]:
tw<-beta/sqrt(betaVar)
print(tw)

(Intercept)    Diametro      Altura 
  0.4511057  23.2086583   5.3772547 


<p style="text-align: justify;font-size: 16px">Calculando e visualizando os p-valores do teste de Wald para avaliação da significância individual de $\beta_j$, $j\in\{0,1,\cdots,p\}$.</p>

In [11]:
pvalor<-2*(1-pnorm(abs(tw)))
print(pvalor)

 (Intercept)     Diametro       Altura 
6.519133e-01 0.000000e+00 7.563013e-08 


<p style="text-align: justify;font-size: 16px">Tanto o p-valor correspondente ao coeficiente da variável <b>Diâmetro</b>, quanto o p-valor correspondente ao coeficiente da variável <b>Altura</b> são menores do que 5%. Portanto, ao nível de 5% significância, o teste de Wald indica que ambos os coeficientes são significativamente diferentes de zero.</p>

<p style="text-align: justify;font-size: 16px">Calculando e visualizando os intervalos de confiança, de acordo com a fórmula
$$IC(\beta_j;1-\alpha)=\left(\hat\beta_j-z_{\left(1-\frac{\alpha}{2}\right)}\sqrt{\widehat{Var(\hat\beta_j)}};\ \hat\beta_j+z_{\left(1-\frac{\alpha}{2}\right)}\sqrt{\widehat{Var(\hat\beta_j)}}\right).$$
</p>

In [12]:
alpha=0.05 #Valor de alpha correspondente ao intervalo com nível de confiança 1-alpha
z<-qnorm(1-alpha/2) #Quantil de ordem 1-alpha/2 da distribuição N(0,1)
IC<-cbind(beta-z*sqrt(betaVar),beta+z*sqrt(betaVar)) #Calculando os intervlos de confiança.
print(IC)

                   [,1]       [,2]
(Intercept) -0.30874032 0.49334948
Diametro     0.13301224 0.15755014
Altura       0.01053538 0.02262038


<p style="text-align: justify;font-size: 16px">Os intervalos de confiança, com 95% de confiança, dos coeficientes das variáveis <b>Diâmetro</b> e <b>Altura</b> não contêm o zero. Pode-se concluir que, ao nível de 5% significância, o teste de Wald indica que ambos os coeficientes são significativamente diferentes de zero.</p>

### 2.5 Executando o teste de Wald utilizando a função ``summary``.

<p style="text-align: justify;font-size: 16px">A função <font face="'Courier New'">summary</font> realiza o teste de Wald para avaliação da significância individual dos coeficientes de regressão. Os dois principais argumentos da função são os seguintes:</p>
    <ol style="text-align: justify; font-size:16px">
        <li><font face="'Courier New'">object</font>: modelo ajustado. Saída da função <font face="'Courier New'">glm</font>.</li>
        <li><font face="'Courier New'">phi</font>: estimativa de máxima veroossimilhança de $\phi$.</li>
    </ol>
</p>

In [13]:
summary(object=aj,dispersion=phi)


Call:
glm(formula = Volume ~ Diametro + Altura, family = Gamma(link = "log"), 
    data = dados)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-0.17812  -0.08826  -0.01317   0.05635   0.17091  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) 0.092305   0.204619   0.451    0.652    
Diametro    0.145281   0.006260  23.209  < 2e-16 ***
Altura      0.016578   0.003083   5.377 7.56e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for Gamma family taken to be 0.008455011)

    Null deviance: 8.31720  on 30  degrees of freedom
Residual deviance: 0.26247  on 28  degrees of freedom
AIC: 151.01

Number of Fisher Scoring iterations: 4


<p style="text-align: justify;font-size: 16px">Para cada $j\in\{0,1,\cdots,p\}$, tem-se que
        <ol style="text-align: justify; font-size:16px">
        <li>a coluna <font face="'Courier New'">Estimate</font> contém o valor de $\hat\beta_j$;</li>
        <li>a coluna <font face="'Courier New'">Std. Error</font> contém o valor de $\widehat{Var(\hat\beta_j)}$;</li>
            <li>a coluna <font face="'Courier New'">z value</font> contém o valor de $t_{W_j}$;</li>
            <li>a coluna <font face="'Courier New'">Pr(>|z|)</font> contém o p-valor do teste de Wald para $\beta_j$.</li>
    </ol>
</p>

<p style="text-align: justify;font-size: 16px">Tanto o p-valor correspondente ao coeficiente da variável <b>Diâmetro</b>, quanto o p-valor correspondente ao coeficiente da variável <b>Altura</b> são menores do que 5%. Portanto, ao nível de 5% significância, o teste de Wald indica que ambos os coeficientes são significativamente diferentes de zero.</p>

### 2.6 Calculando os intervalos de confiança utilizando a função ``confint.default``.

<p style="text-align: justify;font-size: 16px">A função <font face="'Courier New'">confint.default</font> calcula os intervalos de confiança para os coeficientes de regressão, aproximando a distribuição da quantidade $\frac{\hat\beta_j-\beta}{\widehat{Var(\hat\beta_j)}}$ pela distribuição $N(0,1)$. Os dois principais argumentos dessa função são:</p>
    <ol style="text-align: justify; font-size:16px">
        <li><font face="'Courier New'">object</font>: modelo ajustado. Saída da função <font face="'Courier New'">glm</font>.</li>
        <li><font face="'Courier New'">level</font>: confiança do intervalo. Valor de $1-\alpha$.</li>
    </ol>
</p>

In [14]:
confint.default(object=aj,level=0.95)

,2.5 %,97.5 %
(Intercept),-0.33078822,0.51539738
Diametro,0.13233774,0.15822464
Altura,0.01020319,0.02295258


<p style="text-align: justify;font-size: 16px">Os intervalos de confiança, com 95% de confiança, dos coeficientes das variáveis <b>Diâmetro</b> e <b>Altura</b> não contêm o zero. Pode-se concluir que, ao nível de 5% significância, o teste de Wald indica que ambos os coeficientes são significativamente diferentes de zero.</p>

<p style="text-align: justify;font-size: 16px">Os intervalos acima são diferentes daqueles calculados manualmente, na seção 2.4 desse notebook. Isso se deve ao fato de que a função <font face="'Courier New'">confint.default</font> usa o estimador de Pearson para $\phi$, ao invés do estimador de máxima verossimilhança (ver a fórmula 41, no tópico 16 da aula 6). Portanto, para obter, manualmente, os intervalos de confiança acima, deve-se realizar o procedimento descrito na seção 2.4 desse notebook, com a estimativa de Pearson de $\phi$ no lugar da estimativa de máxima verossimilhança.</p>

<p style="text-align: justify;font-size: 16px">Para obter a estimativa de Pearson de $\phi$, utilize o comando <font face="'Courier New'">summary(object)&#36;dispersion</font>, no qual o argumento <font face="'Courier New'">object</font> é o modelo ajustado, ou seja, é a saída da função <font face="'Courier New'">glm</font>.</p>

In [15]:
phi<-summary(aj)$dispersion
print(phi)

[1] 0.009410213


<p style="text-align: justify;font-size: 16px">Agora, realize o procedimento descrito na seção 2.4 desse notebook, com a estimativa de Pearson de $\phi$ no lugar da estimativa de máxima verossimilhança.</p>

<p style="text-align: justify;font-size: 16px">Calculando $Var(\beta_j)$, para cada $j\in\{0,1,\cdots,p\}$. O vetor <font face="'Courier New'">betaVar</font> contém as variâncias calculadas.</p>

In [16]:
Kbb<-phi^{-1}*t(X)%*%X #Calcuando Kbb
KbbInv<-solve(Kbb) #Calcuando a inversa de Kbb
betaVar<-diag(KbbInv) #Recolhendo a diagonal da inversa de Kbb, cujos valores correspondem às variâncias

<p style="text-align: justify;font-size: 16px">Calculando e visualizando os intervalos de confiança. Os intervalos calculados são armazenados na matriz <font face="'Courier New'">IC</font>.</p>

In [17]:
alpha=0.05 #Valor de alpha correspondente ao intervalo com nível de confiança 1-alpha
z<-qnorm(1-alpha/2) #Quantil de ordem 1-alpha/2 da distribuição N(0,1)
IC<-cbind(beta-z*sqrt(betaVar),beta+z*sqrt(betaVar)) #Calculando os intervlos de confiança.
print(IC)

                   [,1]       [,2]
(Intercept) -0.33078822 0.51539738
Diametro     0.13233774 0.15822464
Altura       0.01020319 0.02295258
